# 误差反向传播法

## 示例

### 示例1（乘法）

![image1](../assets/images/d-b-04-001.jpg "苹果")

购买两个苹果，税率10%

In [7]:
# 乘法层的实现
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forword(self,x ,y):
        self.x = x
        self.y = y
        out = x*y

        return out

    def backword(self, dout):
        dx = dout * self.y
        dy = dout * self.x

        return dx, dy

    
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

#正向传播
apple_price = mul_apple_layer.forword(apple, apple_num)
price = mul_tax_layer.forword(apple_price, tax)
print(price)

#反向传播
dprice = 1
dapple_price, dtax = mul_tax_layer.backword(dprice)
dapple, dapple_num = mul_apple_layer.backword(dapple_price)
print(dapple, dapple_price, dtax)



200
220.00000000000003
2.2 1.1 200


### 示例2

![image2](../assets/images/d-b-04-002.jpg "苹果")

购买两个苹果和三个橘子，税率10%

In [8]:
# 加法层的实现
class AddLayer:
    def __init__(self):
        pass

    def forword(self,x ,y):
        out = x+y
        return out

    def backword(self, dout):
        dx = dout * 1
        dy = dout * 1

        return dx, dy

    
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

#正向传播
apple_price = mul_apple_layer.forword(apple, apple_num)
orange_price = mul_orange_layer.forword(orange, orange_num)
all_price = add_apple_orange_layer.forword(apple_price, orange_price)
price = mul_tax_layer.forword(all_price, tax)
print(price)

#反向传播
dprice = 1
dall_price, dtax = mul_tax_layer.backword(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backword(dall_price)
aorange, dorange_num = mul_orange_layer.backword(dorange_price)
dapple, dapple_num = mul_apple_layer.backword(dapple_price)
print(dapple, dapple_num, aorange, dorange_num, dtax)

715.0000000000001
2.2 110.00000000000001 3.3000000000000003 165.0 650


## 激活函数层

### ReLu层

激活函数ReLU可以由如下函数是表示:

$ 
y = 
\begin{cases}  
x  (x > 0) \\
0  (x \leq 0)
\end{cases}
$

求导，


$ 
\frac{d}{dx}y =
\begin{cases}  
1  (x > 0) \\
0  (x \leq 0)
\end{cases}
$


In [10]:
import numpy as np

class Relu:

    def __init__(self):
        self.mask = None

    def forword(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backword(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx
    
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

mask = (x <= 0)
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


### Sigmoid层

激活函数ReLU可以由如下函数是表示:

$ y = \frac{1}{ 1 + {exp(-x)}}$



![image2](../assets/images/d-b-04-003.jpg "sigmoid反向传播")


$ 
\frac{\partial L}{\partial y} y^2 exp(-x) 
= \frac{\partial L}{\partial y} \frac{1}{(1 + {exp(-x)})^2} exp(-x) 
= \frac{\partial L}{\partial y} \frac{1}{1 + {exp(-x)}} \frac{exp(-x)}{1 + {exp(-x)}} 
= \frac{\partial L}{\partial y} y (1-y)
$



In [2]:
import numpy as np

class Sigmoid:

    def __init__(self):
        self.out = None

    def forword(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backword(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

[[ 1.  -0.5]
 [-2.   3. ]]


## Affine/Softmax层

In [ ]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None

    def forword(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    
    def backword(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis = 0)
        return dx